Cleaning up the CSV's so they have same headers


Most code form https://www.youtube.com/watch?v=ViaGirGFJZY&t=259s


except merging the 2 csv and the prompt feature


In [30]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [31]:
og_2024_data = pd.read_csv("/content/drive/MyDrive/NFL data/pbp-2024.csv")

og_2024_scores = pd.read_csv("/content/drive/MyDrive/NFL data/raw_2024_scores.csv")

In [32]:
# Convert the 'GameDate' format in play-by-play data to match the 'Date' format in scores dataset
og_2024_data['GameDate'] = pd.to_datetime(og_2024_data['GameDate']).dt.strftime('%m/%d/%Y')

# Retry matching games for 2022 season using the updated method
# makes a new df with the matched
nfl_play = pd.concat([og_2024_data, og_2024_data])

# Display the first few rows of matched games for 2022 season using the updated method
nfl_play.head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsTwoPointConversion,IsTwoPointConversionSuccessful,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards
0,2024122907,12/29/2024,3,7,3,GB,MIN,1,10,16,...,0,0,CENTER,84,OPP,0,NaN,0,NaN,0
1,2024122907,12/29/2024,3,9,44,MIN,GB,0,0,0,...,0,0,NaN,100,OPP,0,NaN,0,NaN,0
2,2024122907,12/29/2024,3,9,44,MIN,GB,0,0,15,...,0,0,NaN,85,OPP,0,NaN,0,NaN,0
3,2024122907,12/29/2024,3,9,50,MIN,GB,1,10,18,...,0,0,NaN,82,OPP,0,NaN,0,NaN,0
4,2024122907,12/29/2024,3,15,0,MIN,GB,0,0,35,...,0,0,NaN,65,OPP,0,NaN,0,NaN,0


In [33]:
# Changing Scores Data to use Abbreviations
# Extract unique team names from scores data
unique_visitor_teams_2024 = og_2024_scores['Visitor'].unique()
unique_home_teams_2024 = og_2024_scores['Home'].unique()


# Combine all unique teams
all_unique_teams = set(unique_visitor_teams_2024).union(set(unique_home_teams_2024))

# Extract unique team abbreviations from play-by-play data
unique_teams_abbrev_2024 = set(nfl_play['OffenseTeam'].dropna().unique()).union(set(nfl_play['DefenseTeam'].dropna().unique()))


# Combine all unique abbreviations
all_unique_abbrev = unique_teams_abbrev_2024

all_unique_teams, all_unique_abbrev

({'Arizona Cardinals',
  'Atlanta Falcons',
  'Baltimore Ravens',
  'Buffalo Bills',
  'Carolina Panthers',
  'Chicago Bears',
  'Cincinnati Bengals',
  'Cleveland Browns',
  'Dallas Cowboys',
  'Denver Broncos',
  'Detroit Lions',
  'Green Bay Packers',
  'Houston Texans',
  'Indianapolis Colts',
  'Jacksonville Jaguars',
  'Kansas City Chiefs',
  'Las Vegas Raiders',
  'Los Angeles Chargers',
  'Los Angeles Rams',
  'Miami Dolphins',
  'Minnesota Vikings',
  'New England Patriots',
  'New Orleans Saints',
  'New York Giants',
  'New York Jets',
  'Philadelphia Eagles',
  'Pittsburgh Steelers',
  'San Francisco 49ers',
  'Seattle Seahawks',
  'Tampa Bay Buccaneers',
  'Tennessee Titans',
  'Washington Commanders'},
 {'ARI',
  'ATL',
  'BAL',
  'BUF',
  'CAR',
  'CHI',
  'CIN',
  'CLE',
  'DAL',
  'DEN',
  'DET',
  'GB',
  'HOU',
  'IND',
  'JAX',
  'KC',
  'LA',
  'LAC',
  'LV',
  'MIA',
  'MIN',
  'NE',
  'NO',
  'NYG',
  'NYJ',
  'PHI',
  'PIT',
  'SEA',
  'SF',
  'TB',
  'TEN',
  '

In [34]:
# Mapping of team names to abbreviations
team_mapping = {
    'Arizona Cardinals': 'ARI',
    'Atlanta Falcons': 'ATL',
    'Baltimore Ravens': 'BAL',
    'Buffalo Bills': 'BUF',
    'Carolina Panthers': 'CAR',
    'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN',
    'Cleveland Browns': 'CLE',
    'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN',
    'Detroit Lions': 'DET',
    'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU',
    'Indianapolis Colts': 'IND',
    'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC',
    'Las Vegas Raiders': 'LV',
    'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LA',
    'Miami Dolphins': 'MIA',
    'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE',
    'New Orleans Saints': 'NO',
    'New York Giants': 'NYG',
    'New York Jets': 'NYJ',
    'Philadelphia Eagles': 'PHI',
    'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF',
    'Seattle Seahawks': 'SEA',
    'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN',
    'Washington Commanders': 'WAS'
}

# Replace team names with abbreviations in scores data
og_2024_scores['Visitor'] = og_2024_scores['Visitor'].map(team_mapping)
og_2024_scores['Home'] = og_2024_scores['Home'].map(team_mapping)


# Check the updated scores data for both years
new_scores = og_2024_scores


Merging the 2 stat sheets

In [35]:
def make_key(date, team1, team2):
    teams = sorted([team1, team2])
    return f"{pd.to_datetime(date).strftime('%Y-%m-%d')}_{teams[0]}_{teams[1]}"

new_scores['JoinKey'] = new_scores.apply(
    lambda row: make_key(row['Date'], row['Visitor'], row['Home']),
    axis=1
)

nfl_play['JoinKey'] = nfl_play.apply(
    lambda row: make_key(row['GameDate'], row['OffenseTeam'], row['DefenseTeam']),
    axis=1
)

merged = nfl_play.merge(new_scores, on='JoinKey', how='left')
merged.drop(columns='JoinKey', inplace=True)
merged.to_csv('merged_nfl_data.csv', index=False)
merged.head(30)

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,IsNoPlay,PenaltyType,PenaltyYards,Week,Date,Visitor,VisitorScore,Home,HomeScore,OT
0,2024122907,12/29/2024,3,7,3,GB,MIN,1,10,16,...,0,NaN,0,Week 17,12/29/2024,GB,25,MIN,27,NaN
1,2024122907,12/29/2024,3,9,44,MIN,GB,0,0,0,...,0,NaN,0,Week 17,12/29/2024,GB,25,MIN,27,NaN
2,2024122907,12/29/2024,3,9,44,MIN,GB,0,0,15,...,0,NaN,0,Week 17,12/29/2024,GB,25,MIN,27,NaN
3,2024122907,12/29/2024,3,9,50,MIN,GB,1,10,18,...,0,NaN,0,Week 17,12/29/2024,GB,25,MIN,27,NaN
4,2024122907,12/29/2024,3,15,0,MIN,GB,0,0,35,...,0,NaN,0,Week 17,12/29/2024,GB,25,MIN,27,NaN
5,2024122907,12/29/2024,2,2,0,GB,MIN,0,0,0,...,0,NaN,0,Week 17,12/29/2024,GB,25,MIN,27,NaN
6,2025010401,01/04/2025,2,5,32,PIT,CIN,0,0,0,...,0,NaN,0,Week 18,01/04/2025,CIN,19,PIT,17,NaN
7,2025010401,01/04/2025,2,6,33,PIT,CIN,0,0,0,...,0,NaN,0,Week 18,01/04/2025,CIN,19,PIT,17,NaN
8,2025010401,01/04/2025,2,6,37,CIN,PIT,4,1,37,...,0,NaN,0,Week 18,01/04/2025,CIN,19,PIT,17,NaN
9,2025010401,01/04/2025,2,7,20,CIN,PIT,3,6,42,...,0,NaN,0,Week 18,01/04/2025,CIN,19,PIT,17,NaN


In [36]:
# Adding HomeWon Column
import pandas as pd
import numpy as np

# Now, let's proceed with adding the "HomeWon" column
merged['HomeWon'] = merged['HomeScore'] > merged['VisitorScore']

# Display first few rows of updated data for verification
merged[['Date', 'Home', 'Visitor', 'HomeScore', 'VisitorScore', 'HomeWon']].head(), merged[['Date', 'Home', 'Visitor', 'HomeScore', 'VisitorScore', 'HomeWon']].head()

(         Date Home Visitor  HomeScore  VisitorScore  HomeWon
 0  12/29/2024  MIN      GB         27            25     True
 1  12/29/2024  MIN      GB         27            25     True
 2  12/29/2024  MIN      GB         27            25     True
 3  12/29/2024  MIN      GB         27            25     True
 4  12/29/2024  MIN      GB         27            25     True,
          Date Home Visitor  HomeScore  VisitorScore  HomeWon
 0  12/29/2024  MIN      GB         27            25     True
 1  12/29/2024  MIN      GB         27            25     True
 2  12/29/2024  MIN      GB         27            25     True
 3  12/29/2024  MIN      GB         27            25     True
 4  12/29/2024  MIN      GB         27            25     True)

Enter 2 team abrivations below

In [ ]:
import pandas as pd
import numpy as np

# Load the datasets for the 2022 and 2023 seasons that include a "HomeWon" column.
# This column indicates if the home team won the game (1 for win, 0 for loss).

# NOTE: You'll probably have to change these file paths to the paths for your environment
# (click folder icon on left, drag and drop them in, and then right click them to get the path and paste them in the quotes below)
all_data = merged


# Load the dataset containing the upcoming games schedule.
#upcoming_games = pd.read_csv("/content/drive/MyDrive/NFL data/BEST New_Upcoming_Schedule.csv")
hometeam = input("Enter Home Team Abbreviation: ")
visitorteam = input("Enter Visitor Team Abbreviation: ")
upcoming_games = pd.DataFrame({'Home': [hometeam], 'Visitor': [visitorteam]})


# 1. Average Points Scored
# Calculate the average points scored by the home and visitor teams.
avg_points_scored_home = all_data.groupby('Home')['HomeScore'].mean()
avg_points_scored_visitor = all_data.groupby('Visitor')['VisitorScore'].mean()

# 2. Average Points Allowed
# Calculate the average points allowed by the home and visitor teams.
avg_points_allowed_home = all_data.groupby('Home')['VisitorScore'].mean()
avg_points_allowed_visitor = all_data.groupby('Visitor')['HomeScore'].mean()

# Calculate the overall average points scored and allowed by combining the home and visitor averages.
overall_avg_points_scored = (avg_points_scored_home + avg_points_scored_visitor) / 2
overall_avg_points_allowed = (avg_points_allowed_home + avg_points_allowed_visitor) / 2

# 3. Win Rate
# Calculate the total number of wins for home and visitor teams.
home_wins = all_data.groupby('Home')['HomeWon'].sum()
visitor_wins = all_data.groupby('Visitor').apply(lambda x: len(x) - x['HomeWon'].sum())

# Calculate the total number of games played by each team as home and visitor.
total_games_home = all_data['Home'].value_counts()
total_games_visitor = all_data['Visitor'].value_counts()

# Calculate the overall number of wins and total games played by each team.
overall_wins = home_wins + visitor_wins
total_games = total_games_home + total_games_visitor

# Calculate the win rate for each team.
win_rate = overall_wins / total_games

# Calculate the average outcome of games between each pair of teams (home vs visitor).
# head_to_head = all_data.groupby(['Home', 'Visitor'])['HomeWon'].mean()

# Create a new data frame to store the features for each team.
team_features = pd.DataFrame({
    'AvgPointsScored': overall_avg_points_scored,
    'AvgPointsAllowed': overall_avg_points_allowed,
    'WinRate': win_rate
})

# Reset the index of the team_features DataFrame and rename the index column to "Team".
team_features.reset_index(inplace=True)
team_features.rename(columns={'Home': 'Team'}, inplace=True)

# Display the first few rows of the team_features DataFrame.
team_features.head(33)

Enter Home Team Abbreviation: NE


In [38]:
# Examining the columns and a few rows of the datasets to ensure correct structure
upcoming_games.head()

,Home,Visitor
0,JAX,ATL


In [39]:
# Calculate defensive features for each NFL team.

# 1. Average points defended:
# This metric is essentially the same as AvgPointsAllowed, which we already computed in previous steps so we won't recompute it here.

# 2. Average conceded plays:
# A play is considered successful for the offense if it results in a touchdown or doesn't result in a turnover.
# Create a new column 'SuccessfulPlay' in the all_data DataFrame to represent this.
all_data['SuccessfulPlay'] = all_data['IsTouchdown'] | (~all_data['IsInterception'] & ~all_data['IsFumble'])

# Calculate the average rate of successful plays conceded when playing at home.
avg_conceded_plays_home = all_data.groupby('Home')['SuccessfulPlay'].mean()

# Calculate the average rate of successful plays conceded when playing as a visitor.
avg_conceded_plays_visitor = all_data.groupby('Visitor')['SuccessfulPlay'].mean()

# Calculate the overall average rate of successful plays conceded for each team.
overall_avg_conceded_plays = (avg_conceded_plays_home + avg_conceded_plays_visitor) / 2

# 3. Average forced turnovers:
# Create a new column 'Turnover' that indicates if a play resulted in a turnover (either interception or fumble).
all_data['Turnover'] = all_data['IsInterception'] | all_data['IsFumble']

# Calculate the average rate of turnovers forced when playing at home.
avg_forced_turnovers_home = all_data.groupby('Home')['Turnover'].mean()

# Calculate the average rate of turnovers forced when playing as a visitor.
avg_forced_turnovers_visitor = all_data.groupby('Visitor')['Turnover'].mean()

# Calculate the overall average rate of turnovers forced for each team.
overall_avg_forced_turnovers = (avg_forced_turnovers_home + avg_forced_turnovers_visitor) / 2

# Create a new DataFrame to store the defensive features for each team.
team_features_defensive = pd.DataFrame({
    'Team': team_features['Team'].values,
    'AvgPointsDefended': team_features['AvgPointsAllowed'].values,
    'AvgConcededPlays': overall_avg_conceded_plays.values,
    'AvgForcedTurnovers': overall_avg_forced_turnovers.values
})

# Merge the defensive features with the original team features to create a combined DataFrame.
team_features_combined = team_features.merge(team_features_defensive, on='Team')

# Display the first few rows of the combined team features DataFrame.
team_features_combined.head()

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers
0,ARI,23.396436,22.733489,0.457546,22.733489,-1.020181,0.021772
1,ATL,23.120159,25.137638,0.472835,25.137638,-1.021514,0.022980
2,BAL,30.091801,21.502370,0.678042,21.502370,-1.014326,0.016056
3,BUF,30.886875,21.776342,0.745873,21.776342,-1.018697,0.020044
4,CAR,20.533577,31.505754,0.307503,31.505754,-1.017596,0.018268


In [40]:
# Calculate additional offensive features

# 1. Average yards per play
avg_yards_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_per_play = (avg_yards_per_play_home + avg_yards_per_play_visitor) / 2

# 2. Average total yards per game
total_yards_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_per_game = (total_yards_per_game_home + total_yards_per_game_visitor).groupby(level=1).mean()

# 3. Average pass completion rate
avg_pass_completion_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
avg_pass_completion_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
overall_avg_pass_completion_rate = (avg_pass_completion_rate_home + avg_pass_completion_rate_visitor) / 2

# 4. Average touchdowns per game
avg_touchdowns_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_per_game = (avg_touchdowns_per_game_home + avg_touchdowns_per_game_visitor).groupby(level=1).mean()

# 5. Average rush success rate
avg_rush_success_rate_home = all_data.groupby('Home').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
avg_rush_success_rate_visitor = all_data.groupby('Visitor').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
overall_avg_rush_success_rate = (avg_rush_success_rate_home + avg_rush_success_rate_visitor) / 2

# Creating a dataframe for the new offensive features
new_offensive_features = pd.DataFrame({
    'Team': team_features_combined['Team'],
    'AvgYardsPerPlay': overall_avg_yards_per_play.values,
    'AvgYardsPerGame': overall_avg_yards_per_game.values,
    'AvgPassCompletionRate': overall_avg_pass_completion_rate.values,
    'AvgTouchdownsPerGame': overall_avg_touchdowns_per_game.values,
    'AvgRushSuccessRate': overall_avg_rush_success_rate.values
})

# Merging with the existing combined features
team_features_expanded = team_features_combined.merge(new_offensive_features, on='Team')

team_features_expanded.head()


<ipython-input-40-9720dcbc2acb>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
<ipython-input-40-9720dcbc2acb>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
<ipython-input-40-9720dcbc2acb>:24

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate
0,ARI,23.396436,22.733489,0.457546,22.733489,-1.020181,0.021772,3.995585,7.991171,0.887399,0.056276,5.222833
1,ATL,23.120159,25.137638,0.472835,25.137638,-1.021514,0.022980,4.108988,8.217977,0.888102,0.060988,4.825960
2,BAL,30.091801,21.502370,0.678042,21.502370,-1.014326,0.016056,4.133998,8.267995,0.876289,0.069258,5.092409
3,BUF,30.886875,21.776342,0.745873,21.776342,-1.018697,0.020044,4.006057,8.012114,0.878053,0.073643,4.839956
4,CAR,20.533577,31.505754,0.307503,31.505754,-1.017596,0.018268,3.889028,7.778056,0.878550,0.066200,5.156799


In [41]:
# Calculate additional defensive features

# 1. Average yards allowed per play
avg_yards_allowed_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_allowed_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_allowed_per_play = (avg_yards_allowed_per_play_home + avg_yards_allowed_per_play_visitor) / 2

# 2. Average total yards allowed per game
total_yards_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_allowed_per_game = (total_yards_allowed_per_game_home + total_yards_allowed_per_game_visitor).groupby(level=1).mean()

# 3. Average pass completion allowed rate
avg_pass_completion_allowed_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
avg_pass_completion_allowed_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
overall_avg_pass_completion_allowed_rate = (avg_pass_completion_allowed_rate_home + avg_pass_completion_allowed_rate_visitor) / 2

# 4. Average touchdowns allowed per game
avg_touchdowns_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_allowed_per_game = (avg_touchdowns_allowed_per_game_home + avg_touchdowns_allowed_per_game_visitor).groupby(level=1).mean()

# 5. Average rush success allowed rate
avg_rush_success_allowed_rate_home = all_data.groupby('Home').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
avg_rush_success_allowed_rate_visitor = all_data.groupby('Visitor').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
overall_avg_rush_success_allowed_rate = (avg_rush_success_allowed_rate_home + avg_rush_success_allowed_rate_visitor) / 2

# Creating a dataframe for the new defensive features
new_defensive_features = pd.DataFrame({
    'Team': team_features_expanded['Team'],
    'AvgYardsAllowedPerPlay': overall_avg_yards_allowed_per_play.values,
    'AvgYardsAllowedPerGame': overall_avg_yards_allowed_per_game.values,
    'AvgPassCompletionAllowedRate': overall_avg_pass_completion_allowed_rate.values,
    'AvgTouchdownsAllowedPerGame': overall_avg_touchdowns_allowed_per_game.values,
    'AvgRushSuccessAllowedRate': overall_avg_rush_success_allowed_rate.values
})

# Merging with the existing combined features
team_features_complete = team_features_expanded.merge(new_defensive_features, on='Team')

team_features_complete


<ipython-input-41-51e6e5358dd9>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_allowed_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
<ipython-input-41-51e6e5358dd9>:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_allowed_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
<ipython-input-41-

,Team,AvgPointsScored,AvgPointsAllowed,WinRate,AvgPointsDefended,AvgConcededPlays,AvgForcedTurnovers,AvgYardsPerPlay,AvgYardsPerGame,AvgPassCompletionRate,AvgTouchdownsPerGame,AvgRushSuccessRate,AvgYardsAllowedPerPlay,AvgYardsAllowedPerGame,AvgPassCompletionAllowedRate,AvgTouchdownsAllowedPerGame,AvgRushSuccessAllowedRate
0,ARI,23.396436,22.733489,0.457546,22.733489,-1.020181,0.021772,3.995585,7.991171,0.887399,0.056276,5.222833,3.995585,7.991171,0.887399,0.056276,5.222833
1,ATL,23.120159,25.137638,0.472835,25.137638,-1.021514,0.022980,4.108988,8.217977,0.888102,0.060988,4.825960,4.108988,8.217977,0.888102,0.060988,4.825960
2,BAL,30.091801,21.502370,0.678042,21.502370,-1.014326,0.016056,4.133998,8.267995,0.876289,0.069258,5.092409,4.133998,8.267995,0.876289,0.069258,5.092409
3,BUF,30.886875,21.776342,0.745873,21.776342,-1.018697,0.020044,4.006057,8.012114,0.878053,0.073643,4.839956,4.006057,8.012114,0.878053,0.073643,4.839956
4,CAR,20.533577,31.505754,0.307503,31.505754,-1.017596,0.018268,3.889028,7.778056,0.878550,0.066200,5.156799,3.889028,7.778056,0.878550,0.066200,5.156799
5,CHI,18.190665,22.007645,0.287528,22.007645,-1.018362,0.018974,3.631321,7.262642,0.873054,0.053156,4.579084,3.631321,7.262642,0.873054,0.053156,4.579084
6,CIN,28.293845,26.128074,0.530849,26.128074,-1.019615,0.022405,3.944511,7.889023,0.873282,0.075840,4.576071,3.944511,7.889023,0.873282,0.075840,4.576071
7,CLE,15.367091,25.709587,0.175131,25.709587,-1.018939,0.021012,3.652922,7.305844,0.857404,0.055738,4.663375,3.652922,7.305844,0.857404,0.055738,4.663375
8,DAL,20.940253,27.199685,0.416132,27.199685,-1.024597,0.025499,3.841752,7.683503,0.871158,0.059731,4.706710,3.841752,7.683503,0.871158,0.059731,4.706710
9,DEN,24.670987,18.973227,0.556695,18.973227,-1.016647,0.019168,3.713447,7.426893,0.871032,0.053889,4.357228,3.713447,7.426893,0.871032,0.053889,4.357228


In [42]:
# Reload the upcoming games data
upcoming_games = pd.DataFrame({'Home': [hometeam], 'Visitor': [visitorteam]})
# Feature encoding: merging the upcoming games data with the team features data
upcoming_encoded_home = upcoming_games.merge(team_features_complete, left_on='Home', right_on='Team', how='left')
upcoming_encoded_both = upcoming_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

In [43]:
upcoming_encoded_both

,Home,Visitor,Team_Home,AvgPointsScored_Home,AvgPointsAllowed_Home,WinRate_Home,AvgPointsDefended_Home,AvgConcededPlays_Home,AvgForcedTurnovers_Home,AvgYardsPerPlay_Home,...,AvgYardsPerPlay_Visitor,AvgYardsPerGame_Visitor,AvgPassCompletionRate_Visitor,AvgTouchdownsPerGame_Visitor,AvgRushSuccessRate_Visitor,AvgYardsAllowedPerPlay_Visitor,AvgYardsAllowedPerGame_Visitor,AvgPassCompletionAllowedRate_Visitor,AvgTouchdownsAllowedPerGame_Visitor,AvgRushSuccessAllowedRate_Visitor
0,JAX,ATL,JAX,19.20434,25.574488,0.225118,25.574488,-1.014877,0.016138,3.94496,...,4.108988,8.217977,0.888102,0.060988,4.82596,4.108988,8.217977,0.888102,0.060988,4.82596


In [44]:
# Calculate the difference in features as this might be a more predictive representation
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
    upcoming_encoded_both[f'Diff_{col}'] = upcoming_encoded_both[f'{col}_Home'] - upcoming_encoded_both[f'{col}_Visitor']

# Selecting only the difference columns and the teams for clarity
upcoming_encoded_final = upcoming_encoded_both[['Home', 'Visitor'] + [col for col in upcoming_encoded_both.columns if 'Diff_' in col]]

upcoming_encoded_final

,Home,Visitor,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,JAX,ATL,-3.915819,0.436851,-0.247717,0.436851,0.006637,-0.006842,-0.164028,-0.328056,-0.016888,-0.004564,-0.269635,-0.164028,-0.328056,-0.016888,-0.004564,-0.269635


In [45]:
# Prepare training data

# Merge play-by-play data with team features for home teams
training_encoded_home = all_data.merge(team_features_complete, left_on='Home', right_on='Team', how='left')
# Merge the result with team features for visitor teams
training_encoded_both = training_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

# Calculate the difference in features
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
    training_encoded_both[f'Diff_{col}'] = training_encoded_both[f'{col}_Home'] - training_encoded_both[f'{col}_Visitor']

# Filtering out the required columns
training_data = training_encoded_both[[col for col in training_encoded_both.columns if 'Diff_' in col]]
training_labels = training_encoded_both['HomeWon']

In [46]:
training_data.head()

,Diff_AvgPointsScored,Diff_AvgPointsAllowed,Diff_WinRate,Diff_AvgPointsDefended,Diff_AvgConcededPlays,Diff_AvgForcedTurnovers,Diff_AvgYardsPerPlay,Diff_AvgYardsPerGame,Diff_AvgPassCompletionRate,Diff_AvgTouchdownsPerGame,Diff_AvgRushSuccessRate,Diff_AvgYardsAllowedPerPlay,Diff_AvgYardsAllowedPerGame,Diff_AvgPassCompletionAllowedRate,Diff_AvgTouchdownsAllowedPerGame,Diff_AvgRushSuccessAllowedRate
0,-1.553415,-0.256876,0.189113,-0.256876,0.000607,0.000534,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295
1,-1.553415,-0.256876,0.189113,-0.256876,0.000607,0.000534,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295
2,-1.553415,-0.256876,0.189113,-0.256876,0.000607,0.000534,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295
3,-1.553415,-0.256876,0.189113,-0.256876,0.000607,0.000534,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295
4,-1.553415,-0.256876,0.189113,-0.256876,0.000607,0.000534,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295,-0.165618,-0.331235,-0.014417,-0.003846,-0.299295


In [47]:
training_data.shape

(106566, 16)

In [48]:
training_labels.head()

,HomeWon
0,True
1,True
2,True
3,True
4,True


In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize the logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Evaluate the model's performance using cross-validation
cross_val_scores = cross_val_score(logreg, training_data, training_labels, cv=5)

cross_val_scores_mean = cross_val_scores.mean()

cross_val_scores_mean


np.float64(0.7754818960609013)

In [50]:
# Checking the shape of the training data
training_data.shape

(106566, 16)

In [51]:
# Checking for NaN values in the training data
nan_columns = training_data.columns[training_data.isna().any()].tolist()

# Displaying columns with NaN values and the number of NaN values in them
nan_counts = training_data[nan_columns].isna().sum()
nan_counts


,0


In [52]:
# Exclude rows with NaN values from the training data and labels
training_data_cleaned = training_data.dropna()
training_labels_cleaned = training_labels.loc[training_data_cleaned.index]

# Checking the shape of the cleaned data
training_data_cleaned.shape, training_labels_cleaned.shape


((106566, 16), (106566,))

In [53]:
# Re-evaluate the model's performance using cross-validation on the cleaned data
cross_val_scores_cleaned = cross_val_score(logreg, training_data_cleaned, training_labels_cleaned, cv=5)

cross_val_scores_cleaned_mean = cross_val_scores_cleaned.mean()

cross_val_scores_cleaned_mean


np.float64(0.7754818960609013)

In [54]:
# Train the logistic regression model on the entire cleaned training dataset
logreg.fit(training_data_cleaned, training_labels_cleaned)

LogisticRegression(max_iter=1000)

In [55]:
# Predict the probability of the home team winning for the upcoming games
upcoming_game_probabilities = logreg.predict_proba(upcoming_encoded_final[[col for col in upcoming_encoded_final.columns if 'Diff_' in col]])

In [56]:
upcoming_game_probabilities

array([[0.76249083, 0.23750917]])

In [57]:
# Extract the probability that the home team will win (second column of the result)
upcoming_game_prob_home_win = upcoming_game_probabilities[:, 1]

# Add the predictions to the upcoming games dataframe
upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win

# Sort by the probability of the home team winning for better visualization
upcoming_predictions = upcoming_encoded_final[['Home', 'Visitor', 'HomeWinProbability']].sort_values(by='HomeWinProbability', ascending=False)

upcoming_predictions

<ipython-input-57-45adeb99a5ae>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win


,Home,Visitor,HomeWinProbability
0,JAX,ATL,0.237509


Player Stats

In [29]:
player = pd.read_csv("/content/drive/MyDrive/NFL data/play_by_play_2024.csv")
player.head(20)

<ipython-input-29-62bc9434058f>:1: DtypeWarning: Columns (179,180,182,183,189,190,197,198,203,204,205,206,213,214,218,219,220,222,224,226,233,234,235,236,237,238,248,249,253,254,255,260,262,263,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  player = pd.read_csv("/content/drive/MyDrive/NFL data/play_by_play_2024.csv")


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,NaN,NaN,NaN,...,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,0.257819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,-0.200602,NaN,NaN,NaN,NaN,NaN,0.456761,-45.676103
3,83,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,2.028874,1.345418,9.321221,8.0,0.509778,0.363807,0.576656,42.334431
4,108,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,0.754242,0.882798,5.783560,4.0,0.668478,0.255140,0.426443,57.355690
5,133,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,-0.029602,NaN,NaN,NaN,NaN,NaN,0.225364,-22.536427
6,155,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,-0.247749,NaN,NaN,NaN,NaN,NaN,0.424099,-42.409936
7,177,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,-0.530139,NaN,NaN,NaN,NaN,NaN,0.675323,-67.532253
8,199,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,1.680800,0.249492,4.449025,3.0,0.983669,0.983669,0.964793,3.520733
9,224,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0,0,-0.467625,0.408758,2.438554,1.0,1.000000,0.998114,0.392493,60.750705
